In [1]:
from pycalphad import Database, variables as v
from pycalphad.variables import Species

dbf = Database('Al-Cu-Zr_Zhou.tdb')

In [2]:
import itertools
import string
import sympy
import numpy as np

def genericize(mapping, const_array):
    new_array = []
    for subl in const_array:
        new_array.append(tuple(mapping[x] for x in subl))
    return tuple(new_array)
def sigfigs(x, n):
    """Round x to n significant digits"""
    if x != 0:
        return np.around(x, -(np.floor(np.log10(np.abs(x)))).astype(np.int) + (n - 1))
    else:
        return x
def average_piecewise(pw):
    return sum(x.expr for x in pw.args)/len(pw.args)

def categorize_phase(phase_name, constituents, site_ratios):
    if phase_name == 'LIQUID' and len(constituents) == 1:
        return 'liquid'
    elif phase_name == 'GAS' and len(constituents) == 1:
        return 'gas'
    elif len(constituents) == 1 or \
        (len(constituents) == 2 and len(constituents[1]) == 1 and list(constituents[1])[0] == Species('VA')):
        return 'solid'
    elif all(len(x) == 1 for x in constituents):
        return 'intermetallic'
    else:
        return 'complex'

def extract_coefficients(expr, polys):
    result = {}
    for poly in polys:
        if poly == 1:
            continue
        else:
            coef = expr.coeff(poly)
        result[poly] = coef
        expr = expr - poly*coef
    expr = expr.n()
    if isinstance(expr, (sympy.Float, sympy.Integer)):
        result[1] = expr
        expr = sympy.S.Zero
    if expr is not sympy.S.Zero:
        raise ValueError(expr)
    return tuple(result[p] for p in polys)

total_results = []

for combo in itertools.combinations(dbf.elements - {'VA', '/-'}, 2):
    combo_str = sorted(set(combo) | {'VA'})
    generic_combo = list(string.ascii_uppercase[:len(combo)])
    combo = [Species(x) for x in combo_str]
    combo_dict = dict(zip([c for c in combo if c.number_of_atoms > 0], string.ascii_uppercase))
    combo_dict[Species('VA')] = 'VA'
    allowed_phases = set()
    # TODO: Categorize each phase? solid solution, intermetallic, complex intermetallic, liquid, etc.
    for phase_name, phase_obj in dbf.phases.items():
        include_phase = True
        for sublattice in phase_obj.constituents:
            if len(set(sublattice).intersection(combo)) == 0:
                # None of the components in a sublattice are active
                # We cannot build a model of this phase
                include_phase = False
                break
        if include_phase:
            allowed_phases |= {phase_name}
    for param in dbf._parameters.all():
        if param['phase_name'] in allowed_phases and param['parameter_type'] in ['G', 'L']:
            active_pure_elements = [list(x.constituents.keys()) for subl in param['constituent_array'] for x in subl]
            active_pure_elements = [el.upper() for constituents in active_pure_elements for el in constituents]
            active_pure_elements = set(active_pure_elements)
            if not active_pure_elements.issubset(combo_str):
                continue
            ratio_sum = sum(dbf.phases[param['phase_name']].sublattices)
            reduced_ratios = tuple(sigfigs(c, 3) for c in dbf.phases[param['phase_name']].sublattices)
            param_val = param['parameter'].subs(dbf.symbols).subs(dbf.symbols)
            # If parameter is Piecewise, average it
            pw_atoms = param_val.atoms(sympy.Piecewise)
            pw_dict = {key: average_piecewise(key) for key in pw_atoms}
            param_val = param_val.subs(pw_dict)
            polys = [v.T*sympy.log(v.T), sympy.log(v.T)] + [v.T**n for n in range(-30,0)] + [v.T**n for n in range(1,30)] + [1]
            columns = ['system', 'phase_name', 'phase_type', 'site_ratios',
                       'parameter_order', 'constituent_array', 'parameter_value'] + \
                        [str(x) for x in polys]
            id_tuple = (tuple(c for c in combo_str if c != 'VA'), param['phase_name'],
                        categorize_phase(param['phase_name'], dbf.phases[param['phase_name']].constituents, reduced_ratios),
                        reduced_ratios, param['parameter_order'],
                        genericize(combo_dict, param['constituent_array']),
                        param_val) + extract_coefficients(param_val, polys)
            total_results.append(id_tuple)

In [3]:
import pandas as pd
from IPython.display import display

df = pd.DataFrame(total_results, columns=columns)
multi_df = df.drop(columns=['system', 'phase_name', 'parameter_value']).set_index(['phase_type', 'site_ratios', 'constituent_array', 'parameter_order'])
display(multi_df)

T*log(T)  \
phase_type    site_ratios  constituent_array parameter_order                      
liquid        (1.0,)       ((A,),)           0                -19.7745254000000   
                           ((B,),)           0                -16.6258213333333   
                           ((A, B),)         0                                0   
                                             1                                0   
                                             2                                0   
                                             3                                0   
complex       (2.0, 1.0)   ((A,), (A,))      0                -17.7562161000000   
                           ((A,), (B,))      0                -32.9236868000000   
                           ((A,), (A, B))    0                                0   
intermetallic (5.0, 8.0)   ((A,), (B,))      0                -133.177243000000   
complex       (9.0, 11.0)  ((A,), (B,))      0                -208.273348600000   
                           ((B,), (B,))      0                -92.4873200000000   
                           ((A, B), (B,))    1                                0   
intermetallic (0.23, 0.77) ((A,), (B,))      0                -9.84414344200000   
solid         (1.0, 3.0)   ((A,), (VA,))     0                -5.91873870000000   
                           ((B,), (VA,))     0                -4.62436600000000   
                           ((A, B), (VA,))   0                                0   
                                             1                                0   
              (1.0, 1.0)   ((A,), (VA,))     0                -11.8374774000000   
complex       (9.0, 2.0)   ((B,), (A,))      0                -106.913542800000   
solid         (1.0, 1.0)   ((A,), (VA,))     0                -11.8374774000000   
complex       (1.0, 1.0)   ((A,), (B,))      0                -21.0862094000000   
                           ((B,), (B,))      0                -18.4974640000000   
                           ((A, B), (B,))    0                                0   
                           ((A,), (B,))      0                -21.0862094000000   
                           ((B,), (B,))      0                -18.4974640000000   
                           ((A, B), (B,))    1                                0   
intermetallic (0.48, 0.52) ((A,), (B,))      0                -10.4913297920000   
solid         (1.0, 1.0)   ((A,), (VA,))     0                -11.8374774000000   
                           ((B,), (VA,))     0                -9.24873200000000   
...                                                                         ...   
complex       (4.0, 5.0)   ((A,), (B,))      0                -82.8047429333333   
              (1.0, 1.0)   ((A,), (B,))      0                -18.4884440666667   
              (1.0, 2.0)   ((A,), (B,))      0                -27.3394107333333   
              (1.0, 3.0)   ((A,), (B,))      0                -36.1903774000000   
solid         (1.0, 3.0)   ((A,), (VA,))     0                -5.91873870000000   
                           ((B,), (VA,))     0                -15.9031131111111   
                           ((A, B), (VA,))   0                                0   
                                             1                                0   
                                             2                                0   
              (1.0, 1.0)   ((A,), (VA,))     0                -11.8374774000000   
complex       (10.0, 7.0)  ((A,), (B,))      0                -158.864040666667   
              (2.0, 1.0)   ((A,), (B,))      0                -28.1004214666667   
              (51.0, 14.0) ((A,), (B,))      0                -649.392880733333   
              (5.0, 7.0)   ((A,), (B,))      0                -110.144153666667   
              (8.0, 3.0)   ((A,), (B,))      0                -105.651219200000   
solid         (1.0, 1.0)   ((A,), (VA,))     0                -11.8374774000000   
comp

In [4]:
for system_name, system_df in df.groupby('system'):
    system_df = system_df.drop_duplicates(subset='phase_name')
    print('-')
    print('-'.join(system_name))
    print(system_df['phase_type'].value_counts().to_string())

-
AL-CU
complex          7
solid            5
intermetallic    3
liquid           1
-
AL-ZR
complex    20
solid       6
liquid      1
-
CU-ZR
complex    25
solid       4
liquid      1
